# k.LAB API

This Jupyter notebook is a tutorial for the k.LAB API. It is meant for users that want to investigate a bit what can be done with just access to the API.

In this tutorial python is used to handle http requests and deal with results.

## Requirements

### k.LAB subscription

To use this tutorial you might want to use the k.LAB instance setup by the [Integratedmodelling Partnership](https://integratedmodelling.org/). You can get an account from the [their Hub](https://integratedmodelling.org/hub/).
Once you have obtained a username and password, you are good to proceed.

### Python packages

The tutorial requires the following Python packages (you can install them with `pip install <package_name>`):

* requests
* datetime
* json
* ipyleaflet
* shapely

In [ ]:
import requests
import json
from ipyleaflet import Map, Polygon, ImageOverlay, GeoJSON
from datetime import datetime
from shapely.geometry import Polygon as ShplyPolygon

## Base URLs

The base URL for the k.LAB API is:

In [ ]:
BASE_URL = "https://developers.integratedmodelling.org/modeler"
API_BASE = f"{BASE_URL}/api/v2"
PUBLIC_BASE = API_BASE + "/public"

## Endpoints

### Authentication

The login endpoint is accessed at URL:

In [ ]:
loginUrl = API_BASE + "/users/log-in"

it requires a POST request with the following parameters:

In [ ]:
jsonRequestBody = {
   "username": "your username",
   "password": "your password"
}

and returns a JSON response with the following fields:

```json
{
   "redirect": "/modeler/ui/viewer?session=[session]",
   "session": "[session]",
   "publicApps": [...],
   "authorization": "[authorization]"
}
```

The session and authorization fields are essential for user authentication in the k.LAB platform. They should be included in the headers as follows:

```
Authentication: [authorization value]
Authorization: [session value]
```

NOTE: The authentication value is assigned to the **Authentication** header and the session value is assigned to the **Authorization** header, which can be misleading. This behavior cannot be changed currently.

In [ ]:
# do the login call
response = requests.post(loginUrl, json=jsonRequestBody)

# # get the authorization and session from the response
authorization = response.json()["authorization"]
session = response.json()["session"]

# set the authorization header
headers = {
    "Authentication": authorization,
    "Authorization": session
}

### Create a spatio-temporal context

Observations are made in a specific spatio-temporal region of interest referred to as a context. 

The context endpoint is accessed at URL:

In [ ]:
createContextUrl = PUBLIC_BASE + "/submit/context"

The url is accessed via a POST request containing the following parameters:
```json
{
  "urn": null,
  "geometry": "τ0(1){ttype=LOGICAL,period=[1609459200000 1640995200000],tscope=1.0,tunit=YEAR}S2(934,631){bbox=[-75.2281407807369 -72.67107290964314 3.5641500380320963 5.302943221927137],shape=000000000005C...A0987400C8361185B1480,proj=EPSG:4326}",
  "contextType": "earth:Region",
  "observables": [],
  "scenarios": [],
  "estimate": false,
  "estimatedCost": -1
}
```

that can be summarized as follows:

* **urn**: The URN of a known context definition like a geospatial resource (ex. a shapefile). Either urn or geometry is mandatory.
* **geometry**: A string containing a geometry definition. Either urn or geometry is mandatory.
* **contextType**: A string that indicates the type of context. If unsure, us ```earth:Region```.
* **observables**: An array of observables to calculate after the context has been set. Thsis parameter is optional.

Parameters reserved for future use that can be omitted:
* scenarios
* estimate
* estimatedCost


The k.LAB geometry requires additional explanation, as it is has a dedicated format that can be defined using a variety of parameter:

Time: 

* **time**: τ0(1) denotes a generic time with dimension 0 and size 1 while t a specific time. Uppercase letters denote a regular time period, while lowercase letters indicate an irregular one.
* **ttype**: Logical (generic time), physical (specific time), grid or real.
* **period**: Time period as an array of long integers: startMillis, endMillis.
* **tstart**: Starting time as a long integer.
* **tend**: End time as a long integer.
* **tunit**: One of millennium, century, decade, year, month, week, day, hour, minute, second, millisecond, nanosecond, default year.
* **tscope**: integer or floating point number of tunit, default 0

Space:

* **space**: σ indicates generic space, s indicates specific space. Uppercase letters define regular space, lowercase letters an irregular one. S2(934,631) denotes a two-dimensional regular grid consisting of 934 by 631 cells.
* **bbox**: The bounding box of the spatial context represented as an array of decimal numbers: minX, maxX, minY, maxY.
* **sgrid**: The grid resolution in the format: "cellsize unit"
* **shape**: The actual spatial shape in [WKB or WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry). Note that commas must be escaped as ```&comma;```.
* **proj**: the EPSG projection code (defaults to: EPSG:4326)

#### Creating a context using Openstreetmap data

For the purpose of this tutorial and for simplicity (you would not want to create a WKT polygon by hand) let's define the spatial context using the Openstreetmap data.

In [ ]:
# define a place name to search for
placeName = "cantabria"

# use the nominatim API to get the polygon coordinates
urlString = f"https://nominatim.openstreetmap.org/search?q={placeName}&format=geojson&polygon_geojson=1&limit=1"
response = requests.get(urlString)
if response.status_code == 200:    
    data = response.json()
    
    feature = data['features'][0]
    polygonCoordinates = feature['geometry']['coordinates']

    bbox = feature['bbox']
    center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
else:
    print("Error getting data from nominatim")
    

The ipyleaflet package provides a convenient way to visualize maps inside a Jupyter notebook. Let's use that to check the area we are interested in.

In [ ]:
# create a polygon with the coordinates returned by the previous nominatim call 
# for semplicity we use only the first polygon
polygon = Polygon(
    locations=[(y, x) for x, y in polygonCoordinates[0][0]],
    color="red",
    fill_opacity=0
)

m = Map(center=center, zoom=9)
m.add_layer(polygon);
m

#### Submit the context request to the server

With the geometry and other necessary parameters defined, we can submit the context request to the server. 

Since the requested observation might require some time to process, the server returns a response containing a ticket id. That id can be used to check the status of the processing.

The response contains the following important fields:

* id: ID of the ticket to be used to request information
* postDate: the epoch in milliseconds when the request was submitted
* status: The status of the process, which can be OPEN, PENDING or RESOLVED
* type: the type of request (ex. ContextObservation)

In [ ]:
# define the necessary variables
fromYear = "2023-01-01"
toYear = "2024-01-01"
gridResolution = "1000 m"
# convert the OSM coordinates in a shapely polygon to get the wkt. Then replace the comma as required.
shapeWkt = ShplyPolygon(polygonCoordinates[0][0]).wkt.replace(",", "&comma;")

# get epoch time in milliseconds
fromEpoch = int(datetime.strptime(fromYear, "%Y-%m-%d").timestamp() * 1000)
toEpoch = int(datetime.strptime(toYear, "%Y-%m-%d").timestamp() * 1000)

# use string substitution to create the geometry
geometry = "T1{period=[%s %s],tscope=1.0,tunit=YEAR}S2{sgrid=%s,shape=EPSG:4326 %s}" % \
    (fromEpoch, toEpoch, gridResolution, shapeWkt)
createContextRequestBody = {
    "contextType": "earth:Region",
    "geometry": geometry,
}

response = requests.post(createContextUrl, json=createContextRequestBody, headers=headers)
if response.status_code == 200:
    ticketId = response.json()["id"]
    status = response.json()["status"]
    print("Context created with ticket id:", ticketId)
    print("Status of the process: ", status)
else:
    print("Error creating context")
    print(response.json())

#### Check the status of the processing

With the ticket obtained it is possible to check the status of the processing. It is necessary to poll the ticket info endpoint until the status is marked as RESOLVED. The context is then ready to be used.

The ticket info can be obtained by submitting a GET request to the following URL:

In [ ]:
ticketInfoUrl = PUBLIC_BASE + "/ticket/info/" + ticketId

The ticket info request returns a JSON response with the following fields (only important fields are shown and described):

```json
{
   "id": "the ID of the ticket",
   "status": "RESOLVED", 
   "data": {
      "context": "id of context to use for next requests",
      "geometry": "the sent geometry",
      "user": "user",
      "email": "email of the user",
      "artifacts": "if the context message contained observables, the ids of resolved artifacts"
   }
}

```

In [ ]:
# make a ticket info call
status = "EMPTY"

iteration = 1
# wait until the process is resolved
while status != "RESOLVED":
    response = requests.get(ticketInfoUrl, headers=headers)
    if response.status_code == 200:
        status = response.json()["status"]
        if status == "RESOLVED":
            # get the context id
            contextId = response.json()["data"]["context"]
            print("Context of id", contextId, "created successfully")
            break
        else:
            print("Status of the process at iteration", iteration, "=", status )
        
        iteration += 1
    else:
        print("Error getting ticket info")
        print(response.json())
        break

### Make a raster observation

Now that we have a context, we can issue observations in it. The observation endpoint is accessed at URL:

In [ ]:
makeObservationUrl = PUBLIC_BASE + "/submit/observation/" + contextId

The request has to be a POST and contain the observable URN. 

To observe for example the slope of the terrain, we can use the following request body:

```json
{
 "urn": "geography:Slope",
}
```

In [ ]:
observation = "landcover:LandCoverType"

response = requests.post(makeObservationUrl, json={"urn":observation}, headers=headers)
if response.status_code == 200:
    observationTicketId = response.json()["id"]
    observationStatus = response.json()["status"]
    print("Observation submitted with ticket id:", observationTicketId)
    print("Status of the process: ", observationStatus)
else:
    print("Error creating observation")
    print(response.json())

As with the context, the server returns a ticket id that can be used to check the status of the processing. 

In [ ]:
observationTicketInfoUrl = PUBLIC_BASE + "/ticket/info/" + observationTicketId

# make a ticket info call
status = "EMPTY"

iteration = 1
# wait until the process is resolved
while status != "RESOLVED":
    response = requests.get(observationTicketInfoUrl, headers=headers)
    if response.status_code == 200:
        status = response.json()["status"]
        if status == "RESOLVED":
            # get the context id
            artifactsIds = response.json()["data"]["artifacts"]
            print(" Created artifacts", artifactsIds, "created successfully")
            break
        else:
            print("Status of the process at iteration", iteration, "=", status )
        
        iteration += 1
    else:
        print("Error getting ticket info")
        print(response.json())
        break

#### Export the observation results

Once the observation is completed, the results can be exported in a variety of formats. The export endpoint is accessed at URL:

In [ ]:
exportDataUrl = PUBLIC_BASE + "/export/data/" + artifactsIds

An observation can be exported in a varaiety of formats, depending on the data type. For example, in the case of a raster, the result can be esported as a png (mimetype image/png) or as a GeoTIFF (mimetype image/tiff).

The request has to be a POST and the format is driven by the mimetype used in the accept header. For example, to export the previously observed raster result into an image we can use the following request:

In [ ]:
exportHeaders = headers.copy()
exportHeaders["Accept"] = "image/png"

response = requests.get(exportDataUrl, headers=exportHeaders)
if response.status_code == 200:
    # get the image from the response and show it
    imageBytes = response.content

    # save the image to a file
    with open("image.png", "wb") as f:
        f.write(imageBytes)
else:
    print("Error getting the image resource.")
    print(response.json())


Once the image is saved to disk, we can display it together with the initially set spatial context.

In [ ]:
# set min lat/lon and max lat/lon from the bbox
min_lat = bbox[1]
min_lon = bbox[0]
max_lat = bbox[3]
max_lon = bbox[2]

image_overlay = ImageOverlay(url="./image.png", bounds=((min_lat, min_lon), (max_lat, max_lon)))

m = Map(center=center, zoom=9)
m.add_layer(image_overlay)
m.add_layer(polygon)
m

When dealing with raster data a legend comes in handy, expecially when the data is categorical. The legend can be exported as follows:

In [ ]:
exportLegendUrl = PUBLIC_BASE + "/export/legend/" + artifactsIds

exportHeaders = headers.copy()
exportHeaders["Accept"] = "application/json"

response = requests.get(exportLegendUrl, headers=exportHeaders)
if response.status_code == 200:
    legendMap = response.json()
else:
    print("Error getting the legend.")
    print(response.json())

def createLegend(items):
    legend_html = ""
    
    for color, label in items:
        box_style = f"width: 20px; height: 20px; background-color: {color}; margin-right: 5px;"
        legend_html += f'<div style="display: flex; align-items: center; margin-bottom: 10px;">'
        legend_html += f'<div style="{box_style}"></div>'
        legend_html += f'<span>{label}</span>'
        legend_html += '</div>'
    
    return legend_html

colors = legendMap["colors"]
labels = legendMap["labels"]

legendItems = list(zip(colors, labels))
legendHtml = createLegend(legendItems)

from IPython.display import HTML
HTML(legendHtml)

### Make a vector observation

Now let's try to make an observation that returns a vector result. 

In [ ]:
observation = "infrastructure:Town"

response = requests.post(makeObservationUrl, json={"urn":observation}, headers=headers)
if response.status_code == 200:
    observationTicketId = response.json()["id"]
    observationStatus = response.json()["status"]
    print("Observation submitted with ticket id:", observationTicketId)
    print("Status of the process: ", observationStatus)
else:
    print("Error creating observation")
    print(response.json())

observationTicketInfoUrl = PUBLIC_BASE + "/ticket/info/" + observationTicketId

# make a ticket info call
status = "EMPTY"

iteration = 1
# wait until the process is resolved
while status != "RESOLVED":
    response = requests.get(observationTicketInfoUrl, headers=headers)
    if response.status_code == 200:
        status = response.json()["status"]
        if status == "RESOLVED":
            # get the context id
            artifactsIds = response.json()["data"]["artifacts"]
            print(" Created artifacts", artifactsIds, "created successfully")
            break
        else:
            print("Status of the process at iteration", iteration, "=", status )
        
        iteration += 1
    else:
        print("Error getting ticket info")
        print(response.json())
        break

exportDataUrl = PUBLIC_BASE + "/export/data/" + artifactsIds

exportHeaders = headers.copy()
exportHeaders["Accept"] = "application/json"

response = requests.get(exportDataUrl, headers=exportHeaders)
if response.status_code == 200:
    # get the image from the response and show it
    observationGeojson = response.json()
    observationGeojson = json.dumps(observationGeojson)

    # save the image to a file
    with open("observation.json", "w") as f:
        f.write(observationGeojson)
else:
    print("Error getting the image resource.")
    print(response.json())

Once again, let's have a look at the map view of our observations.

In [ ]:
with open('observation.json', 'r') as f:
    data = json.load(f)

geo_json = GeoJSON(data=data)

m = Map(center=center, zoom=9)
m.add_layer(image_overlay)
m.add_layer(polygon)
m.add_layer(geo_json)
m

### Additional export options

#### Export the dataflow

One of the most important products during the resolution process of an observation, is the dataflow. The dataflow allows to understand the provenance of the resources, being it data or models, used to calculate the result.

It is possible to export the dataflow of an observation using the URL:

In [ ]:
exportDataflowUrl = PUBLIC_BASE + "/export/dataflow/" + artifactsIds

exportHeaders = headers.copy()
exportHeaders["Accept"] = "text/plain"

response = requests.get(exportDataflowUrl, headers=exportHeaders)
if response.status_code == 200:
    dataflow = response.text
    print(dataflow)
else:
    print("Error getting dataflow")
    print(response.text)

#### Export the observation data structure

It is possible to obtain the data structure of the observation by submitting a POST request to the following URL:

In [ ]:
exportDataStructureUrl = PUBLIC_BASE + "/export/structure/" + artifactsIds

exportHeaders = headers.copy()
exportHeaders["Accept"] = "application/json"

response = requests.get(exportDataStructureUrl, headers=exportHeaders)
if response.status_code == 200:
    # print the data structure with indentation
    dataStructure = response.json()
    print(json.dumps(dataStructure, indent=4))
else:
    print("Error getting data structure")
    print(response.json())

The data structure can be useful to have some insight over the dataset without the need to download it. It contains information about available formats but also range values and histograms of the dataset. 